# Online Retail

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Data Cleansing

### Load and Understand Data

Pertama-tama kita load dan read data dari file .csv terlebih dahulu.

In [ ]:
online_retail = pd.read_csv('../input/onlineretail/OnlineRetail.csv', encoding='latin-1', parse_dates=[4])
online_retail.head()

In [ ]:
online_retail.shape

Berikut adalah penjelasan mengenai data di atas :
* InvoiceNo merupakan kolom yang berisi nomor invoice yang diberikan secara unik pada tiap transaksinya. Jika kode   dimulai dengan huruf 'c', ini menandakan pembatalan.
* StockCode merupakan kolom berisikan kode dari masing-masing produk yang diberikan secara unik.
* Description merupakan kolom dari nama-nama produk.
* Quantity adalah jumlah setiap produk (item) per transaksi. Jika value bernilai negatif probably barang tersebut   direturn oleh pembeli.
* InvoiceDate adalah hari dan waktu setiap transaksi dibuat.
* UnitPrice berisikan harga produk per unit dalam sterling.
* CustomerID adalah kolom yang berisikan nomor customer yang diberikan secara unik untuk setiap customernya.
* Country  adalah kolom berisikan nama negara dari customer berasal

### Check for _Null Value_

In [ ]:
online_retail.isnull().sum()

Dari hasil di atas dapat diketahui bahwa kolom `Description` dan `CustomerID` memiliki beberapa missing value. Kita dapat menghapus missing value tersebut.

In [ ]:
online_retail.dropna(inplace=True)
online_retail.reset_index(drop=True, inplace=True)

Cek kembali missing value dari data.

In [ ]:
online_retail.isnull().sum()

In [ ]:
online_retail.head()

In [ ]:
online_retail.info()

### Data Manipulate in Column

Berdasarkan data di atas, kolom `CustomerID` memiliki tipe data numerik yang berupa float. Kita akan mengubah data ini menjadi tipe string guna memudahkan proses modeling (bila ada).

In [ ]:
online_retail['CustomerID'] = online_retail['CustomerID'].astype(str)

In [ ]:
online_retail.dtypes

### Check Outlier Data

In [ ]:
online_retail.head()

#### Quantity

In [ ]:
print('Mean Unit Price : ', online_retail['Quantity'].mean())
print('Median Unit Price : ', online_retail['Quantity'].median())

In [ ]:
sns.distplot(online_retail['Quantity'])
line3 = plt.axvline(online_retail['Quantity'].mean(), color='r', linestyle='--', label='mean')
line4 = plt.axvline(online_retail['Quantity'].median(), color='b', linestyle='--', label='median')
plt.title('Quantity Distribution')
plt.legend(handles=[line3,line4], loc=1)
plt.xlabel('quantity')
plt.ylabel('count')

Berdasarkan grafik distribusi di atas, kita dapat menarik insight bahwa ada beberapa item yang quantitynya di bawah 0 atau bernilai negatif. Seperti yang telah dijelaskan di atas, quantity yang bernilai negatif merupakan barang yang direturn. Mari kita buat DataFrame untuk quantity di bawah 0.

In [ ]:
quantity_under_zero = online_retail[online_retail['Quantity']<0]
quantity_under_zero

#### UnitPrice

In [ ]:
print('Mean Unit Price : ', online_retail['UnitPrice'].mean())
print('Median Unit Price : ', online_retail['UnitPrice'].median())

In [ ]:
sns.distplot(online_retail['UnitPrice'])
line3 = plt.axvline(online_retail['UnitPrice'].mean(), color='r', linestyle='--', label='mean')
line4 = plt.axvline(online_retail['UnitPrice'].median(), color='b', linestyle='--', label='median')
plt.title('Unit Price Distribution')
plt.legend(handles=[line3,line4], loc=1)
plt.xlabel('unit_price')
plt.ylabel('count')

Berdasarkan grafik distribusi `UnitPrice`, terlihat bahwa grafik tersebut memiliki ekor panjang ke kanan. Kita coba buatkan DataFrame sendiri untuk kasus tersebut.

In [ ]:
unit_price_outlier = online_retail[online_retail['UnitPrice']>4]
unit_price_outlier

## Data Analysis

### Country x Numb of Transaction

In [ ]:
online_retail.head()

In [ ]:
country_transaction = online_retail['Country'].value_counts().head()

In [ ]:
sns.barplot(x=country_transaction.index, y=country_transaction.values)
plt.title('TOP 5 Country Transaction')
plt.xlabel('Country')
plt.ylabel('Number of Transaction')

Terdapat gap yang jauh antara UK dengan negara-negara lainnya. Ini adalah suatu hal yang wajar terlebih jika memang pusat penjualannya ada di UK. Sekarang kita coba cek jumlah transaksi di beberapa negara lainnya.

In [ ]:
numb_country = online_retail['Country'].nunique()
print(f'Dari data tersebut ada sebanyak {numb_country} negara.')

country = pd.DataFrame(online_retail['Country'].value_counts()).reset_index()
country.columns = ['Country', 'Numb of Transaction']
country

### Country x Numb of Customer

In [ ]:
online_retail.head()

In [ ]:
country_customer = online_retail[['Country', 'CustomerID']].groupby(['Country'])
country_customer = country_customer.agg({'CustomerID':'nunique'})
country_customer.sort_values(by='CustomerID', ascending=False).reset_index()

Sejalan dengan jumlah transaksi di tiap negaranya, jumlah pembeli di UK, Germany, dan France juga menjadi TOP 3 dari number of customer. Salah satu contoh yang menarik ialah customer yang berasal dari negara EIRE tapi memiliki transaksi sebanyak 7485.

### Returned Goods

In [ ]:
online_retail.head()

Pertama, kita coba cek terlebih dahulu ada berapa jumlah kuantitas barang yang direturn oleh pembeli menggunakan DataFrame yang telah kita buat sebelumnya.

In [ ]:
quantity_return = abs(quantity_under_zero['Quantity'].sum())
print(f'Jumlah barang yang dikembalikan adalah {quantity_return} buah')

Sekarang kita coba lihat secara detail, barang apa saja yang dikembalikan beserta jumlah barangnya.

In [ ]:
quantity_under_zero[['Description', 'Quantity']].groupby(['Description']).sum().sort_values(by=['Quantity']).head()

Data di atas memperlihatkan lima barang terbanyak yang direturn oleh pembeli. Ternyata gap antara barang pertama dan kedua dengan barang ketiga hingga kelima cukup jauh.

### Total Amount

Pada bagian ini kita akan menampilkan grafik time series dari total amount khususnya customer yang berasal dari UK. Langkah pertama yang harus kita lakukan adalah membuang data di mana `Quantity`nya bernilai negatif.

In [ ]:
new_retail_online = online_retail.drop(quantity_under_zero.index, axis=0).reset_index(drop=True)
new_retail_online

Setelah menghilangkan semua row dengan `Quantity` kurang dari nol, selanjutnya kita harus membuat kolom `TotalAmount` yang merupakan hasil perkalian dari `Quantity` dan `UnitPrice`. Lalu kemudian kita bisa membuang kolom `Quantity` dan `UnitPrice`. Terakhir, kita hitung `TotalAmount` di masing-masing `InvoiceDate`nya.

In [ ]:
new_retail_online['TotalAmount'] = new_retail_online['Quantity'] * new_retail_online['UnitPrice']

In [ ]:
new_retail_online.drop(new_retail_online[['Quantity', 'UnitPrice']], axis=1, inplace=True)

Berhubung karena kita akan menghitung jumlah `Total Amount` berdasarkan tanggal, maka kita akan membuat sebuah kolom baru yaitu `Date` dengan data berupa tanggal pembelian dari kolom `InvoiceDate` yang berisikan tanggal dan waktu pembelian.

In [ ]:
new_retail_online['Date'] = [d.date() for d in new_retail_online['InvoiceDate']]
new_retail_online.drop(new_retail_online[['InvoiceDate']], axis=1, inplace=True)
new_retail_online['Date'] = pd.to_datetime(new_retail_online['Date'])

In [ ]:
new_retail_online = new_retail_online[['Date', 'InvoiceNo', 'StockCode', 'Description', 
                                       'CustomerID', 'Country', 'TotalAmount']]
new_retail_online.head()

In [ ]:
new_retail_online = new_retail_online[['Date', 'TotalAmount']].groupby(['Date'])
new_retail_online = new_retail_online.agg({'TotalAmount': 'sum'}).reset_index()
new_retail_online.head()

Terakhir, kita tinggal membuat plottingan grafik line chart dengan y axisnya kolom `TotalAmount` dan x axisnya `InvoiceDate`.

In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(data=new_retail_online, x=new_retail_online['Date'], y=new_retail_online['TotalAmount'])
plt.title('TotalAmount by Date', fontsize=15)
plt.xlabel('Date', fontsize=13)
plt.ylabel('TotalAmount', fontsize=13)

Grafik line chart di atas memperlihatkan pergerakan dari `TotalAmount` yang terus meningkat, terlebih dari bulan Sep hingga bulan Des. Hal ini dibuktikan juga dengan meningkatnya baseline di bulan tersebut. Namun, apakah besarnya `TotalAmount` ini dipengaruhi juga oleh banyaknya `Quantity` dari barang yang dibeli?